# 집계, 분류

#### [ 데이터 준비 ]
#### [ 간단한 집계 연산 ] 
#### [ GroupBy - 분할(split), 적용(apply), 결합(combine) ]
#### [ GroupBy 객체 ] 
#### [ 집계, 필터, 변환, 적용 ]



- 집계 연산 종류
  - sum() : 합
  - mean() : 평군
  - median() : 중간값
  - min() : 최소값 
  - max() : 최대값

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

print("pandas ver : ",pd.__version__)
print("numpy ver : ",np.__version__)
print("seaborn : ",sns.__version__)

import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

pandas ver :  0.24.2
numpy ver :  1.16.4
seaborn :  0.9.0


### [ 데이터 준비 ]
  - Seaborn패키지의 행성 데이터 사용
  - 2014년 까지 발견된 1,000개 이상의 행성 정보

In [2]:
planets = sns.load_dataset('planets')
print(type(planets))
print(planets.shape)
print(planets.ndim)
print(planets.size)

<class 'pandas.core.frame.DataFrame'>
(1035, 6)
2
6210


In [3]:
display('planets.head()','planets.tail()')

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009
,method,number,orbital_period,mass,distance,year
1030,Transit,1,3.941507,NaN,172.0,2006
1031,Transit,1,2.615864,NaN,148.0,2007
1032,Transit,1,3.191524,NaN,174.0,2007
1033,Transit,1,4.125083,NaN,293.0,2008


### [ 간단한 집계 연산]

- **Series에서 집계 연산 예**

In [4]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
print(ser)

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64


In [5]:
ser.sum()

2.811925491708157

In [6]:
ser.mean()

0.5623850983416314

In [7]:
ser.median()

0.5986584841970366

In [8]:
ser.min()

0.15601864044243652

In [9]:
ser.max()

0.9507143064099162

- **Dataframe에서 집계 연산  예**

In [10]:
df = pd.DataFrame({'A':rng.rand(5),
                   'B':rng.rand(5)})
display('df')

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [11]:
df.sum()

A    2.389442
B    2.217101
dtype: float64

In [12]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

In [13]:
df.median()

A    0.601115
B    0.212339
dtype: float64

In [14]:
df.min()

A    0.058084
B    0.020584
dtype: float64

In [15]:
df.max()

A    0.866176
B    0.969910
dtype: float64

- **planets 데이터 를 이용한 집계 연산**
  - planets데이터에  NaN데이터 확인

In [16]:
planets.isna().any(axis=0)

method            False
number            False
orbital_period     True
mass               True
distance           True
year              False
dtype: bool

- NaN 데이터를 제외한 나머지 데이터들의 전반적인 속성을 확인

In [17]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000



| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | 항목전체 개수                       |
| ``first()``, ``last()``  | 첫 항목과 마지막 항목                |
| ``mean()``, ``median()`` | 평균값과 중앙값                     |
| ``min()``, ``max()``     | 최소값과 최대값                     |
| ``std()``, ``var()``     | 표준편차와 분산                     |
| ``mad()``                | 절대 평균 편차                     |
| ``prod()``               | 전체 항목의 곱                     |
| ``sum()``                | 전체 항목의 합                     |

___

### [ GroupBy - 분할(split), 적용(apply), 결합(combine) ]

- **분할** : 지정된 키 값을 기준으로 Dataframe을 나누고 분류 
- **적용** : 개별 그룹 내에서 일반적으로 집계, 변환 필터링 같은 함수를 계산
- **결합** : 연산의 결과를 결과 배열에 병합

In [18]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])

display('df')

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


- **`groupby('column name')`** : 원하는 키 열(column) 이름을 전달해 가장 기본적인 분할 적용 결합 연산을 수행
- Dataframe
  - `DataFrame.groupby(self, by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=False, observed=False, **kwargs)`
- Series
  - `Series.groupby(self, by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=False, observed=False, **kwargs)`

In [19]:
groupby_df=df.groupby('key')
print(groupby_df)
print(type(groupby_df))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


In [20]:
# DataFrameGroupBy 객체는 Dataframe의 일종의 View라고 생각하면됨
groupby_df.sum()

,data
key,
A,3
B,5
C,7


### [ GroupBy 객체 ] 

- groupby 객체에서 중요한 연산은 집계, 필터, 변환, 적용 임

In [21]:
display('planets')

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300000,7.100,77.40,2006
1,Radial Velocity,1,874.774000,2.210,56.95,2008
2,Radial Velocity,1,763.000000,2.600,19.84,2011
3,Radial Velocity,1,326.030000,19.400,110.62,2007
4,Radial Velocity,1,516.220000,10.500,119.47,2009
5,Radial Velocity,1,185.840000,4.800,76.39,2008
6,Radial Velocity,1,1773.400000,4.640,18.15,2002
7,Radial Velocity,1,798.500000,NaN,21.41,1996
8,Radial Velocity,1,993.300000,10.300,73.10,2008
9,Radial Velocity,2,452.800000,1.990,74.79,2010


- groupby 객체 적용

In [33]:
gb_p = planets.groupby('method')

In [34]:
# 입력데이터의 컬럼 확인
planets.columns

Index(['method', 'number', 'orbital_period', 'mass', 'distance', 'year'], dtype='object')

- 열이름을 참조해 특정 Series 그룹을 선택함 `pandas.core.groupby.generic.SeriesGroupBy`

In [35]:
gb_p['orbital_period']

- groupby 객체를 이용한 중간값 연산

In [40]:
gb_p.median()

,number,orbital_period,mass,distance,year
method,,,,,
Astrometry,1.0,631.180000,NaN,17.875,2011.5
Eclipse Timing Variations,2.0,4343.500000,5.125,315.360,2010.0
Imaging,1.0,27500.000000,NaN,40.395,2009.0
Microlensing,1.0,3300.000000,NaN,3840.000,2010.0
Orbital Brightness Modulation,2.0,0.342887,NaN,1180.000,2011.0
Pulsar Timing,3.0,66.541900,NaN,1200.000,1994.0
Pulsation Timing Variations,1.0,1170.000000,NaN,NaN,2007.0
Radial Velocity,1.0,360.200000,1.260,40.445,2009.0
Transit,1.0,5.714932,1.470,341.000,2012.0


In [38]:
gb_p['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

In [42]:
gb_p.median()['orbital_period']

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

- groupby 객체는 **그룹을 순회**할 수 있도록 지원하며 각 그룹을 Series, Dataframe 으로 반환함

In [45]:
# gb_p = planets.groupby('method')
for (method, group) in gb_p:
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


In [59]:
# planets method 컬럼의 데이터 정보
planets['method'].unique()

array(['Radial Velocity', 'Imaging', 'Eclipse Timing Variations',
       'Transit', 'Astrometry', 'Transit Timing Variations',
       'Orbital Brightness Modulation', 'Microlensing', 'Pulsar Timing',
       'Pulsation Timing Variations'], dtype=object)

- method 컬럼을 기준으로 groupby 객체를 적용하였기 때문에 group정보는 method 컬럼의 데이터가 들어감

In [61]:
gb_p.groups

{'Astrometry': Int64Index([113, 537], dtype='int64'),
 'Eclipse Timing Variations': Int64Index([32, 37, 38, 39, 40, 41, 42, 43, 44], dtype='int64'),
 'Imaging': Int64Index([  29,   30,   31,   33,   34,   35,   36,   47,   54,   68,   69,
               70,   71,   72,   73,   74,   75,   89,   90,  124,  150,  392,
              423,  577,  635,  639,  643,  644,  645,  646,  900,  937,  948,
              949,  950,  957, 1027, 1029],
            dtype='int64'),
 'Microlensing': Int64Index([902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 917, 918,
             919, 920, 921, 922, 923, 924, 925, 926, 927, 928],
            dtype='int64'),
 'Orbital Brightness Modulation': Int64Index([787, 788, 792], dtype='int64'),
 'Pulsar Timing': Int64Index([941, 942, 943, 944, 945], dtype='int64'),
 'Pulsation Timing Variations': Int64Index([958], dtype='int64'),
 'Radial Velocity': Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
             ...
             829, 831, 835, 89

- 디스패치 메서드(Dispatch method) : GroupBy 객체가 명시적으로 구현하지 않은 메서드는 일부 파이썬 클래스 매직을 통해 그 그룹에 전달되고 호출

In [74]:
gb_p['year'].describe().unstack()

       method                       
count  Astrometry                          2.000000
       Eclipse Timing Variations           9.000000
       Imaging                            38.000000
       Microlensing                       23.000000
       Orbital Brightness Modulation       3.000000
       Pulsar Timing                       5.000000
       Pulsation Timing Variations         1.000000
       Radial Velocity                   553.000000
       Transit                           397.000000
       Transit Timing Variations           4.000000
mean   Astrometry                       2011.500000
       Eclipse Timing Variations        2010.000000
       Imaging                          2009.131579
       Microlensing                     2009.782609
       Orbital Brightness Modulation    2011.666667
       Pulsar Timing                    1998.400000
       Pulsation Timing Variations      2007.000000
       Radial Velocity                  2007.518987
       Transit             

### [ 집계, 필터, 변환, 적용 ]

- aggregate()
- filter()
- transform()
- apply()

In [76]:
# 데이터 선언
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


- 집계 - **`aggregate()`** 
  - 문자열, 함수, 리스트 등을 취해 한번에 모든 집계를 계산 함
  - `GroupBy.aggregate(self, func, *args, **kwargs)`

In [96]:
# 각 컬럼마다 최소값 중앙값 최대값 계산
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [97]:
# data1 컬럼은 최소값 계산 지정
# data2 컬럼은 최대값 계산 지정
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


- 필터링 - **`filter()`**
  - 그룹 속성을 기준으로 데이터를 걸러낼 수 있음
  - `DataFrameGroupBy.filter(self, func, dropna=True, *args, **kwargs)`

In [98]:
# 필터링 함수
def filter_func(x):
    return x['data2'].std() > 4

In [99]:
print(type(df.groupby('key').filter(filter_func)))

<class 'pandas.core.frame.DataFrame'>


In [100]:
display('df','df.groupby("key").std()')

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby("key").std()
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

In [101]:
display('df.groupby("key").filter(filter_func)')

,key,data1,data2
1,B,1,0
2,C,2,3
4,B,4,7
5,C,5,9


- 변환 - **`transform()`**
  - 데이터의 재결합을 위해 전체 데이터의 변횐된 버전을 반환함
  - 입력값과 항상 같은 형상(shape)을 가짐
  - `GroupBy.transform(self, func, *args, **kwargs)`

In [102]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


- 변환 - **`apply()`**
  - 임의의 함수를 그룹 결과에 적용할 때 사용 
  - `GroupBy.apply(self, func, *args, **kwargs)`

In [103]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)")

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9
,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3


### [ 분할 키 지정하기 ]

In [106]:
L = [0, 1, 0, 1, 2, 0]

In [108]:
gb_l = df.groupby(L)

In [111]:
display('df', 'df.groupby(L).sum()')

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(L).sum()
   data1  data2
0      7     17
1      4      3
2      4      7

In [110]:
gb_l.groups

{0: Int64Index([0, 2, 5], dtype='int64'),
 1: Int64Index([1, 3], dtype='int64'),
 2: Int64Index([4], dtype='int64')}

- gb_l 의 groups를 통해 그룹핑 됨
  - index 0 : L 리스트의 0,2,5번째 인덱스를 가지는 항목
  - index 1 : L 리스트의 1,3 번째 인덱스를 가지는 항목 
  - index 2 : L 리스트의 4 번째 인덱스를 가지는 항목

- data1 열의 0 인덱스 : 0 + 2 + 5 = 7
- data1 열의 1 인덱스 : 1 + 3 = 4
- data1 열의 2 인덱스 : 4

- data2 열의 0 인덱스 : 5 + 3 + 9 = 17
- data2 열의 1 인덱스 : 0 + 3 =  3
- data2 열의 2 인덱스 : 7

### [ 분류(Grouping) 예제 ] 
- 연대별로 발견한 행성의 갯수 파악

In [123]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0
